# URL Scanner Script - Gad Azriel
## Automation script that interacts with cybersecurity-related APIs (VirusTotal)

## Program Description
This program scans URLs using the VirusTotal API.
It retrieves the scan results, analyzes the data, generates a text report and a pie chart summarizing the detection results, and finally compresses the files into a password-protected ZIP file.



In [ ]:
%%capture
!pip install requests validators matplotlib pyminizip

### Import Required Libraries

In this cell, we import all the necessary libraries used in the program:

- `requests`: For sending HTTP requests to interact with the VirusTotal API.
- `validators`: For validating the URLs entered by the user.
- `matplotlib.pyplot`: For creating visualizations, such as the pie chart summarizing the scan results.
- `pyminizip`: For compressing and password-protecting the generated files.
- `os`: For handling file operations, such as checking if files exist and removing them.


In [ ]:
import requests
import validators
import matplotlib.pyplot as plt
import pyminizip
import os

## Detailed Function Explanations

### `checkAPI()`
This function validates the VirusTotal API key.
- Prompts the user to enter an API key.
- Sends a GET request to the VirusTotal API with the Google URL (https://google.com) to check if the key is valid.
- If the key is valid (status = 200), the function returns the key.
- If the key is not valid, it prompts the user to enter a new key by recursively calling itself.
- If an exception occurs, the function prints the exception and prompts for a new key.

For example, you can use the API key provided to me, and it is:
02130d00ab89cd38a614f60a7f7a81625c0b40317f286992d20fb657a311f8e3

In [ ]:
def checkAPI():
    try:
        apiKey = input("Enter your VirusTotal API key: ")
        url = 'https://www.virustotal.com/vtapi/v2/url/report'
        params = {'apikey': apiKey, 'resource': "https://google.com"}
        response = requests.get(url, params=params)
        if response.status_code == 200: ## If the response status is 200 so the API key is valid
            return apiKey
        else:
            print("That API key does not seem to be valid, please try again.")
            return checkAPI()
    except Exception as e:
        print("An error occurred:", e)
        return checkAPI()

### `checkURL()`
This function validates the URL entered by the user for scanning.
- Prompts the user to enter a URL.
- Checks if the URL is valid and reachable.
- If the URL is not valid or reachable, it prompts the user to enter a new URL by recursively calling itself.
- If an exception occurs, the function prints the exception and prompts for a new URL.

You can check any URL you want.
Example URL: https://www.facebook.com/


In [ ]:
def checkURL():
    try:
        urlInput = input("Enter a URL to scan: ")
        if not urlInput.startswith("http"):
            urlInput = "http://" + urlInput # Add 'http://' if not present
        if validators.url(urlInput):  # Validate the URL
            try:
                response = requests.get(urlInput)
                if response.status_code == 200:
                    return urlInput # Return the URL if reachable
                else:
                    print("The URL is not reachable, please try again.")
                    return checkURL()
            except:
                print("The URL is not reachable, please try again.")
                return checkURL()
        else:
            print("Invalid URL, please try again.")
            return checkURL()
    except Exception as e:
        print("An error occurred:", e)
        return checkURL()

### `performScan(urlInput, apiKey)`
This function performs a scan on the URL using the VirusTotal API.
- Sends a POST request with the URL and the API key to the VirusTotal API.
- Returns the scan ID.


In [ ]:
def performScan(urlInput, apiKey):
    url = 'https://www.virustotal.com/vtapi/v2/url/scan'
    params = {'apikey': apiKey, 'url': urlInput}
    response = requests.post(url, data=params)
    id = response.json().get('scan_id')
    return id

### `report(id, apiKey)`
This function retrieves the scan report from VirusTotal using the scan ID.
- Sends a GET request with the scan ID and the API key to the VirusTotal API.
- Returns the data in JSON format.


In [ ]:
def report(id, apiKey):
    url = 'https://www.virustotal.com/vtapi/v2/url/report'
    params = {'apikey': apiKey, 'resource': id}
    response = requests.get(url, params=params)
    data = response.json() # Convert the response to JSON format
    return data

#### `analyze_data(data, urlInput)`
This function analyzes the scan data and generates insights.
- Identifies the number of positive detections, total scans, and the scan date.
- Identifies engines that detected the URL as malicious or safe.
- Generates a report and calls `report_export` to save the results.

In [ ]:
def analyze_data(data,urlInput):
    positives = data.get('positives', 0) #If the "positives" key doesn't exist in the data dictionary, it assigns the default value 0 to the positives variable.
    total = data.get('total', 0)
    scan_date = data.get('scan_date', '--')
    scans = data.get('scans', {})

    # Analysis for potential threats and anomalies
    suspicious_engines = []
    safe_engines = []
    for engine, result in scans.items():
        if result['detected']:
            suspicious_engines.append(engine) #Engines that detected threats
        else:
            safe_engines.append(engine)
    report_export(scan_date, positives, total, suspicious_engines, scans,urlInput)


### `report_export(scan_date, positives, total, suspicious_engines, scans, urlInput)`
This function generates a text report summarizing scan results.
- Includes details such as the scan date, number of positive detections, and total scans.
- Lists engines that detected the URL as malicious.
- Saves the report to a text file.
- Calls `generate_report` to create a visual summary.

In [ ]:
def report_export(scan_date, positives, total, suspicious_engines, scans,urlInput):
    reportText = "Report Summary\n"
    reportText += f"URL: {urlInput}\n"
    reportText += "==============\n"
    reportText += f"Scan Date: {scan_date}\n"
    reportText += f"Positives: {positives}/{total} (Detected threats/Total scans)\n\n"
    reportText += "\nDetailed Scan Results:\n"

    for engine, result in scans.items():
        reportText += f"    Engine name: {engine}:\n"
        for key, value in result.items():
            reportText += f"        {key}: {value}\n"

    reportText += "\nAnalysis:\n"
    if positives > 0:
        reportText += "Potential Threats Detected!\n"
        reportText += f"{positives} out of {total} scans detected the URL as malicious.\n"
        reportText += "Engines that detected threats:\n"
        for engine in suspicious_engines:
            reportText += f" - {engine}\n"
    else:
        reportText += "No threats detected by any engine. The URL appears to be SAFE.\n"

     # Save report to a file
    with open('report.txt', 'w') as file:
        file.write(reportText)
    generate_report(positives, total,urlInput)

### `generate_report(positives, total, urlInput)`
This function generates a pie chart summarizing threat detection results.
- Visualizes the proportion of safe scans versus detected threats.
- Saves the chart as a PNG image.
- Calls `compress_and_protect_files` to secure the generated files.

In [ ]:
def generate_report(positives, total,urlInput):
    if total == 0:
        print("Error: Total scans is zero, cannot generate report.")
        return
    labels = 'Safe', 'Detected Threats'
    sizes = [total - positives, positives]
    colors = ['green', 'red']
    explode = (0, 0.1)  # Explode the 2nd slice (Detected Threats) for emphasis
    plt.figure(figsize=(10, 6))      # Create a matplotlib figure
    # Generate pie chart
    plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
    plt.axis("equal")   # Set aspect ratio for a circular pie chart
    plt.title(f'Threat Detection Summary\n\nURL: {urlInput}')
    plt.savefig("threat_detection_summary.png")     # Save the chart as a PNG image
    print("The report was exported successfully\n")
    compress_and_protect_files()

### `compress_and_protect_files()`
This function compresses and password-protects the generated files.
- Compresses the text report and pie chart into a password-protected ZIP file.
- Deletes the original files after compression.


In [ ]:
def compress_and_protect_files():
    input_files = ["./report.txt", "./threat_detection_summary.png"]   # Relative paths to the files
    prefixes = ["", ""]  # List of empty prefixes
    output_zip = "./output.zip" # Path for the output zip file
    # If the output file exists, remove it
    if os.path.exists(output_zip):
        os.remove(output_zip)
    password = input("Enter password for zip file: ")
    com_lvl = 5 # Compression level
    try:
        pyminizip.compress_multiple(input_files, prefixes, output_zip, password, com_lvl) # Compress the files and protect with password
        print("Files have been compressed and locked with a password.")
    except ValueError as e:
        print(f"An error occurred: {e}")

    # Delete the original files
    for file_path in input_files:
        if os.path.exists(file_path):
            os.remove(file_path)
        else:
            print(f"The file {file_path} does not exist.")


### Main Function
The main function that runs the entire process:

**For example**, you can use the API key provided to me, and it is: 02130d00ab89cd38a614f60a7f7a81625c0b40317f286992d20fb657a311f8e3

You can check any URL you want.
**Example URL**: https://www.facebook.com/

In [ ]:
apiKey = checkAPI()
urlInput = checkURL()
id = performScan(urlInput, apiKey)
data = report(id, apiKey)
analyze_data(data,urlInput)

© Gad Azriel